In [1]:
import pandas as pd
import json as js

In [2]:
metadata = pd.read_csv('/Users/maurice/Documents/certification/dermdetect/csv/dataset.csv')

In [3]:
metadata = metadata.drop('image_path', axis= 1)

In [4]:
metadata['image_id'] = metadata['image_id'].str.upper()

In [5]:
metadata['image_id'] = metadata['image_id'].str.replace(r'_\d+\.JPG$', '', regex=True)

In [6]:
metadata

,image_id,target
0,ISIC_1064919,benign
1,ISIC_2708737,benign
2,ISIC_1944865,benign
3,ISIC_0032083,benign
4,ISIC_0028959,benign
...,...,...
72595,ISIC_0014041,malignant
72596,ISIC_0011627,malignant
72597,ISIC_0022073,malignant
72598,ISIC_8560106,malignant


In [7]:

# Nombre initial de lignes
initial_count = len(metadata)

# Supprimer les doublons basés sur les colonnes image_id et target
metadata = metadata.drop_duplicates(subset=['image_id', 'target']).reset_index(drop=True)


# Nombre de lignes après suppression des doublons
final_count = len(metadata)

# Calculer le nombre de lignes supprimées
lines_removed = initial_count - final_count

# Afficher le résultat
print(f"Nombre de lignes supprimées : {lines_removed}")
print(metadata)


Nombre de lignes supprimées : 28436
           image_id     target
0      ISIC_1064919     benign
1      ISIC_2708737     benign
2      ISIC_1944865     benign
3      ISIC_0032083     benign
4      ISIC_0028959     benign
...             ...        ...
44159  ISIC_0022566  malignant
44160  ISIC_0027300  malignant
44161  ISIC_0063793  malignant
44162  ISIC_2255411  malignant
44163  ISIC_0035525  malignant

[44164 rows x 2 columns]


In [8]:
import requests
import time
import json

# URL de base de l'API
base_url = "https://api.isic-archive.com/api/v2/images/"

# En-têtes de requête
headers = {
    "accept": "application/json",
    "X-CSRFToken": "rFjRY2QusJwfWoXaHS7zsTd6dpMRFjG3syKTIAB1xH9ivj5GKHNovnjlxY5qoDWO"
}

# Fonction pour récupérer les métadonnées avec retry
def fetch_metadata(image_id, base_url, headers, retries=3, delay=2):
    url = f"{base_url}{image_id}/"
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)  # Timeout ajouté
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Erreur pour {image_id}: {response.status_code}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Tentative {attempt + 1} échouée pour {image_id}: {e}")
            time.sleep(delay)  # Attente avant la prochaine tentative
    print(f"Échec définitif pour {image_id} après {retries} tentatives.")
    return None

# Liste des image_id dans le DataFrame `metadata`
image_ids = metadata["image_id"].tolist()

# Charger les métadonnées existantes si elles sont déjà enregistrées
output_file = "metadata_partial.json"
try:
    with open(output_file, "r") as json_file:
        all_metadata = json.load(json_file)
        # Correction ici : utiliser "isic_id" au lieu de "id"
        fetched_ids = {entry["isic_id"] for entry in all_metadata}
except (FileNotFoundError, json.JSONDecodeError):
    all_metadata = []
    fetched_ids = set()

# Collecter les métadonnées pour tous les image_id restants
for idx, image_id in enumerate(image_ids):
    if image_id in fetched_ids:
        continue  # Sauter les images déjà traitées
    print(f"Fetching metadata for image_id: {image_id} ({idx + 1}/{len(image_ids)})")
    metadata_entry = fetch_metadata(image_id, base_url, headers)
    if metadata_entry:  # Ajouter seulement si la requête est réussie
        all_metadata.append(metadata_entry)
        fetched_ids.add(image_id)

    # Sauvegarder toutes les 100 requêtes
    if len(all_metadata) % 100 == 0:
        with open(output_file, "w") as json_file:
            json.dump(all_metadata, json_file, indent=4)
        print(f"Progress saved after {len(all_metadata)} entries.")

# Sauvegarder les métadonnées finales
with open(output_file, "w") as json_file:
    json.dump(all_metadata, json_file, indent=4)

print(f"Les métadonnées ont été sauvegardées dans '{output_file}'.")


Fetching metadata for image_id: ISIC_1064919 (1/44164)
Fetching metadata for image_id: ISIC_2708737 (2/44164)
Fetching metadata for image_id: ISIC_1944865 (3/44164)
Fetching metadata for image_id: ISIC_0032083 (4/44164)
Fetching metadata for image_id: ISIC_0028959 (5/44164)
Fetching metadata for image_id: ISIC_0022595 (6/44164)
Fetching metadata for image_id: ISIC_0061082 (7/44164)
Fetching metadata for image_id: ISIC_3763008 (8/44164)
Fetching metadata for image_id: ISIC_3650533 (9/44164)
Fetching metadata for image_id: ISIC_4270738 (10/44164)
Fetching metadata for image_id: ISIC_1463406 (11/44164)
Fetching metadata for image_id: ISIC_1134319 (12/44164)
Fetching metadata for image_id: ISIC_2060077 (13/44164)
Fetching metadata for image_id: ISIC_0030724 (14/44164)
Fetching metadata for image_id: ISIC_0032452 (15/44164)
Fetching metadata for image_id: ISIC_4206375 (16/44164)
Fetching metadata for image_id: ISIC_0148465 (17/44164)
Fetching metadata for image_id: ISIC_5688950 (18/44164)
F

In [10]:
# Charger le fichier JSON depuis le chemin donné
json_file_path = "/Users/maurice/Documents/certification/dermdetect/preprocessing/metadata_partial.json"

with open(json_file_path, "r") as file:
    json_data = json.load(file)

# Désimbriquer et convertir en DataFrame
df = pd.json_normalize(json_data)

# Sélectionner et renommer les colonnes souhaitées
df_final = df[[
    "isic_id",
    "copyright_license",
    "files.full.url",
    "files.full.size",
    "files.thumbnail_256.url",
    "files.thumbnail_256.size",
    "metadata.acquisition.pixels_x",
    "metadata.acquisition.pixels_y",
    "metadata.acquisition.image_type",
    "metadata.clinical.concomitant_biopsy",
    "metadata.clinical.sex",
    "metadata.clinical.anatom_site_general",
    "metadata.clinical.benign_malignant",
    "metadata.clinical.diagnosis_1",
    "metadata.clinical.diagnosis_confirm_type",
    "metadata.clinical.age_approx",
    "metadata.clinical.lesion_id",
    "metadata.clinical.patient_id"
]]

# Renommer les colonnes pour plus de clarté
df_final.columns = [
    "isic_id",
    "copyright_licence",
    "full_url",
    "full_size",
    "256_url",
    "256_size",
    "pixels_x",
    "pixels_y",
    "image_type",
    "concomitant_biopsy",
    "sex",
    "anatom_site_general",
    "benign_malignant",
    "diagnosis_1",
    "diagnosis_confirm_type",
    "age_approx",
    "lesion_id",
    "patient_id"
]

# Définir le chemin de sortie pour le fichier CSV
csv_output_path = "/Users/maurice/Documents/certification/dermdetect/csv/metadata.csv"

# Sauvegarder en CSV
df_final.to_csv(csv_output_path, index=False)

# Afficher un message de confirmation avec le chemin du fichier CSV
csv_output_path


'/Users/maurice/Documents/certification/dermdetect/csv/metadata.csv'